In [2]:
import pandas as pd
import numpy as np
import re
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import CPTAC.Endometrial as CPTAC

Welcome to the CPTAC data service package. This import contains
information about the package. In order to access a specific data set,
import a CPTAC subfolder by either 'import CPTAC.DataName' or 'from
CPTAC import DataName'.
Loading Endometrial CPTAC data:
Loading Dictionary...
Loading Clinical Data...
Loading Acetylation Proteomics Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter embargo() to open the webpage for more details.


In [3]:
gene='TP53'
somatic = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
transcriptomics = CPTAC.get_transcriptomics()

# A list of mutations present in the hotspot according to our Hotspot3D output
hotspot_mutations = ['p.P250L', 'p.R248W', 'p.S241C', 'p.241del','p.K164E', 'p.R249S', 'p.R273H', 'p.R282W',
                    'p.S127Y', 'p.G244D', 'p.A159P']
mutated_hotspot = somatic.loc[(somatic['Location'].isin(hotspot_mutations)) & (somatic['Gene'] == gene)]

# Get a list of patient identifiers for those who have mutations in the hotspot
hotspot_patients_both_keys = mutated_hotspot[['Clinical_Patient_Key', 'Patient_Id']]
hotspot_patients = hotspot_patients_both_keys['Clinical_Patient_Key']
hotspot_patients

586      S001
1249     S006
1335     S008
3962     S016
4609     S020
44776    S071
44808    S072
46588    S082
49326    S096
49407    S097
Name: Clinical_Patient_Key, dtype: object

# All TP53 Mutants vs. Wildtype TP53

In [4]:
#bonferroni correction
pcutoff = 0.05 / len(proteomics.columns)

proteomics_all_mut_wt_tstat = {}

# Go through every protein in our proteomics dataframe
for protein in proteomics.columns:
    if (protein != 'idx') and (protein != gene):
        # Create dataframe with mutation status of TP53 with proteomics of the given protein
        p53_mutations_protein = CPTAC.compare_mutations(proteomics, protein, gene)
        # Create a column showing the location of TP53 mutation
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        # Perform the ttest if there is enough data for that protein
        mutated_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] != 'Wildtype']
        wt_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(mutated_tp53) < 2: continue
        ttest = scipy.stats.ttest_ind(mutated_tp53[protein], wt_tp53[protein])
        # Print significant values
        if ttest[1] <= pcutoff:
            proteomics_all_mut_wt_tstat[protein] = ttest[0]

In [5]:
transcriptomics_all_mut_wt_tstat = {}
pcutoff = 0.05 / len(transcriptomics.columns)

# Go through every protein in our proteomics dataframe
for protein in transcriptomics.columns:
    if (protein != 'idx') and (protein != gene):
        # Create dataframe with mutation status of TP53 with proteomics of the given protein
        p53_mutations_protein = CPTAC.compare_mutations(transcriptomics, protein, gene)
        # Create a column showing the location of TP53 mutation
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        # Perform the ttest if there is enough data for that protein
        mutated_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] != 'Wildtype']
        wt_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(mutated_tp53) < 2: continue
        ttest = scipy.stats.ttest_ind(mutated_tp53[protein], wt_tp53[protein])
        # Print significant values
        if ttest[1] <= pcutoff:
            transcriptomics_all_mut_wt_tstat[protein] = ttest[0]

In [33]:
proteomics_shared_tstat = {}
proteomics_unique_tstat = {}

transcriptomics_shared_tstat = {}
transcriptomics_unique_tstat = {}

shared_identical_direction_pos = list()
shared_identical_direction_neg = list()
shared_opposite_direction = list()


for protein in proteomics_all_mut_wt_tstat:
    if protein in transcriptomics_all_mut_wt_tstat:
        proteomics_shared_tstat[protein] = proteomics_all_mut_wt_tstat[protein]
        transcriptomics_shared_tstat[protein] = transcriptomics_all_mut_wt_tstat[protein]

for protein in proteomics_all_mut_wt_tstat:
    if protein not in transcriptomics_all_mut_wt_tstat:
        proteomics_unique_tstat[protein] = proteomics_all_mut_wt_tstat[protein]
        
for protein in transcriptomics_all_mut_wt_tstat:
    if protein not in proteomics_all_mut_wt_tstat:
        transcriptomics_unique_tstat[protein] = transcriptomics_all_mut_wt_tstat[protein]

for protein in proteomics_shared_tstat:
    if proteomics_shared_tstat[protein] > 0 and transcriptomics_shared_tstat[protein] > 0:
        shared_identical_direction_pos.append(protein)
    if proteomics_shared_tstat[protein] < 0 and transcriptomics_shared_tstat[protein] < 0:
        shared_identical_direction_neg.append(protein)
    if proteomics_shared_tstat[protein] < 0 and transcriptomics_shared_tstat[protein] > 0:
        shared_opposite_direction.append(protein)
    if proteomics_shared_tstat[protein] > 0 and transcriptomics_shared_tstat[protein] < 0:
        shared_opposite_direction.append(protein)
        
print('There are ' + str(len(shared_identical_direction_pos)) + ' shared correlations in the same positive direction')
print(shared_identical_direction_pos)

print('\n' + 'There are ' + str(len(shared_identical_direction_neg)) + ' shared correlations in the same negative direction')
print(shared_identical_direction_neg)

print('\n' + 'There are ' + str(len(shared_opposite_direction)) + ' shared correlations in the opposite direction')
print(shared_opposite_direction)

print('\n' + 'There are ' + str(len(proteomics_unique_tstat)) + ' unique proteomic correlations not found in the transcriptome')
print('There are ' + str(len(transcriptomics_unique_tstat)) + ' unique transcriptomic correlations not found in the proteome')

There are 46 shared correlations in the same positive direction
['ABCF1', 'ABT1', 'ATAD2', 'CAD', 'CDKAL1', 'CIP2A', 'CMPK2', 'CMSS1', 'CMTR1', 'CSE1L', 'DDX27', 'EIF2AK2', 'GRB7', 'HELZ2', 'HERC5', 'IFIT1', 'IRF2BPL', 'ISG15', 'KLRG2', 'KRI1', 'LSG1', 'LSM14B', 'MRGBP', 'MTHFD2', 'MX1', 'MX2', 'NGDN', 'NOL10', 'NUP153', 'OGFR', 'PRKCI', 'PRPF6', 'RAE1', 'RIOK1', 'SENP2', 'SLC4A1AP', 'SP110', 'STAT1', 'STAT2', 'TOPBP1', 'TPD52L2', 'UBE2C', 'USP39', 'XPO5', 'YTHDF1', 'ZGPAT']

There are 16 shared correlations in the same negative direction
['ALG2', 'ARFIP1', 'ARFIP2', 'CA8', 'DOCK5', 'EPS8', 'FBXO4', 'IDNK', 'KIF13B', 'KLHDC7A', 'MYO5C', 'NPDC1', 'PTPN3', 'SIAE', 'SNX7', 'TESC']

There are 0 shared correlations in the opposite direction
[]

There are 94 unique proteomic correlations not found in the transcriptome
There are 273 unique transcriptomic correlations not found in the proteome


### Unique Protein List for the Proteome and Transcriptome

In [20]:
unique_prot_all_mut_wt = list()
for protein in proteomics_unique_tstat:
    unique_prot_all_mut_wt.append(protein)

unique_prot_all_mut_wt

['AES',
 'AKAP8L',
 'BIN3',
 'BIRC2',
 'BUB1B',
 'CBS',
 'CDK11B',
 'CDYL',
 'CEBPZ',
 'CHD4',
 'COPS4',
 'CRADD',
 'DDX18',
 'DDX39A',
 'DDX52',
 'DLG5',
 'DLGAP5',
 'ECT2',
 'EIF4G1',
 'ESF1',
 'FBXO22',
 'FMN1',
 'FXR1',
 'GEMIN2',
 'GFM1',
 'GMPR2',
 'GMPS',
 'HEATR1',
 'HMCES',
 'IFIT2',
 'KDM2A',
 'KIF2C',
 'KIFC1',
 'MDC1',
 'MTUS1',
 'NAA25',
 'NCL',
 'NOL11',
 'NOL7',
 'NOM1',
 'OAS3',
 'PA2G4',
 'PAK2',
 'PAXX',
 'PBLD',
 'PDCD11',
 'PICK1',
 'PLCE1',
 'PRRC2A',
 'PSMD5',
 'PTAR1',
 'PTPN2',
 'PUF60',
 'RAB5IF',
 'RABL6',
 'RBBP6',
 'RCC2',
 'RIF1',
 'RRP1B',
 'RRP36',
 'SCRN3',
 'SDF2',
 'SEC16A',
 'SEPSECS',
 'SGSM3',
 'SMG1',
 'SMPD1',
 'TAF2',
 'TOLLIP',
 'TOP1',
 'TOP2A',
 'TPX2',
 'TRAF3IP2',
 'TRIM2',
 'TRIP13',
 'TRMT10C',
 'TRMT6',
 'TTC4',
 'UBR5',
 'USP47',
 'UTP3',
 'UTP4',
 'VPS13C',
 'VPS25',
 'VPS36',
 'VRK1',
 'WDR43',
 'WDR75',
 'XPO1',
 'XRN1',
 'ZC3H8',
 'ZCCHC17',
 'ZNF217',
 'ZNF622']

In [21]:
unique_tran_all_mut_wt = list()
for protein in transcriptomics_unique_tstat:
    unique_tran_all_mut_wt.append(protein)

unique_tran_all_mut_wt

['AAMP',
 'ABCF3',
 'ABHD16B',
 'ABHD18',
 'ACAD9',
 'ACAP2',
 'ADAM17',
 'ADRM1',
 'AEN',
 'AIP',
 'ALG3',
 'ANKFN1',
 'ANKHD1-EIF4EBP3',
 'ANKLE1',
 'ANKRA2',
 'ANKRD27',
 'AP2M1',
 'ARFGAP1',
 'ARFRP1',
 'ARHGAP26-IT1',
 'ARMC8',
 'ARSD',
 'ATAD3B',
 'ATL2',
 'AURKA',
 'BABAM1',
 'BAK1',
 'BBS4',
 'BYSL',
 'C20orf27',
 'C6orf106',
 'C9orf152',
 'CABLES2',
 'CASC4',
 'CCDC124',
 'CDCA8',
 'CDK5RAP1',
 'CDKN1A',
 'CENPA',
 'CENPO',
 'CHRAC1',
 'CKS1B',
 'CLCN2',
 'COG3',
 'COL28A1',
 'COLCA1',
 'COLCA2',
 'COLGALT1',
 'CSNK2A1',
 'CSNK2B',
 'CSTF1',
 'CTCFL',
 'CTNNBL1',
 'CYB5D2',
 'DAXX',
 'DCAF15',
 'DERL1',
 'DHX16',
 'DNAJB11',
 'DNMBP',
 'DPH2',
 'E2F1',
 'E2F3',
 'EEFSEC',
 'EFL1',
 'EHMT2',
 'EIF4A3',
 'ERCC3',
 'ERCC6L2',
 'ERI3',
 'ERICH3',
 'ESPL1',
 'FAAP24',
 'FAM131A',
 'FAM149A',
 'FAM214A',
 'FAM219B',
 'FAM83D',
 'FBXO45',
 'FGF21',
 'FKBPL',
 'FNDC7',
 'FSD1',
 'FYTTD1',
 'GALNT7',
 'GGCX',
 'GID8',
 'GMEB2',
 'GOLGA1',
 'GPATCH3',
 'GPN1',
 'GRAMD2B',
 'GSTM2P1',
 '

# TP53 Hotspot Mutants vs. Wildtype TP53

In [7]:
proteomics_htspt_wt_tstat = {}
pcutoff = 0.05 / len(proteomics.columns)

for protein in proteomics.columns:
    if (protein != 'idx') and (protein != gene):
        # Create dataframe with mutation status of TP53 with proteomics of the given protein
        p53_mutations_protein = CPTAC.compare_mutations(proteomics, protein, gene)
        # Create a column showing the location of TP53 mutation
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        # Perform the ttest
        hotspot_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == True]
        wt_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(hotspot_tp53) < 2: continue
        ttest = scipy.stats.ttest_ind(hotspot_tp53[protein], wt_tp53[protein])
        # Print significant values
        if ttest[1] <= pcutoff:
            proteomics_htspt_wt_tstat[protein] = ttest[0]

In [8]:
transcriptomics_htspt_wt_tstat = {}
pcutoff = 0.05 / len(transcriptomics.columns)

for protein in transcriptomics.columns:
    if (protein != 'idx') and (protein != gene):
        # Create dataframe with mutation status of TP53 with proteomics of the given protein
        p53_mutations_protein = CPTAC.compare_mutations(transcriptomics, protein, gene)
        # Create a column showing the location of TP53 mutation
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        # Perform the ttest
        hotspot_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == True]
        wt_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(hotspot_tp53) < 2: continue
        ttest = scipy.stats.ttest_ind(hotspot_tp53[protein], wt_tp53[protein])
        # Print significant values
        if ttest[1] <= pcutoff:
            transcriptomics_htspt_wt_tstat[protein] = ttest[0]

In [37]:
proteomics_shared_tstat_2 = {}
proteomics_unique_tstat_2 = {}

transcriptomics_shared_tstat_2 = {}
transcriptomics_unique_tstat_2 = {}

shared_identical_direction_pos_2 = list()
shared_identical_direction_neg_2 = list()
shared_opposite_direction_2 = list()


for protein in proteomics_htspt_wt_tstat:
    if protein in transcriptomics_htspt_wt_tstat:
        proteomics_shared_tstat_2[protein] = proteomics_htspt_wt_tstat[protein]
        transcriptomics_shared_tstat_2[protein] = transcriptomics_htspt_wt_tstat[protein]

for protein in proteomics_htspt_wt_tstat:
    if protein not in transcriptomics_htspt_wt_tstat:
        proteomics_unique_tstat_2[protein] = proteomics_htspt_wt_tstat[protein]
        
for protein in transcriptomics_htspt_wt_tstat:
    if protein not in proteomics_htspt_wt_tstat:
        transcriptomics_unique_tstat_2[protein] = transcriptomics_htspt_wt_tstat[protein]

for protein in proteomics_shared_tstat_2:
    if proteomics_shared_tstat_2[protein] > 0 and transcriptomics_shared_tstat_2[protein] > 0:
        shared_identical_direction_pos_2.append(protein)
    if proteomics_shared_tstat_2[protein] < 0 and transcriptomics_shared_tstat_2[protein] < 0:
        shared_identical_direction_neg_2.append(protein)
    if proteomics_shared_tstat_2[protein] < 0 and transcriptomics_shared_tstat_2[protein] > 0:
        shared_opposite_direction_2.append(protein)
    if proteomics_shared_tstat_2[protein] > 0 and transcriptomics_shared_tstat_2[protein] < 0:
        shared_opposite_direction_2.append(protein)
        
print('There are ' + str(len(shared_identical_direction_pos_2)) + ' shared correlations in the same positive direction')
print(shared_identical_direction_pos_2)

print('\n' + 'There are ' + str(len(shared_identical_direction_neg_2)) + ' shared correlations in the same negative direction')
print(shared_identical_direction_neg_2)

print('\n' + 'There are ' + str(len(shared_opposite_direction_2)) + ' shared correlations in the opposite direction')
print(shared_opposite_direction_2)

print('\n' + 'There are ' + str(len(proteomics_unique_tstat_2)) + ' unique proteomic correlations not found in the transcriptome')
print('There are ' + str(len(transcriptomics_unique_tstat_2)) + ' unique transcriptomic correlations not found in the proteome')

There are 19 shared correlations in the same positive direction
['CMSS1', 'CMTR1', 'EIF2AK2', 'HELZ2', 'IFIT2', 'LSG1', 'LSM14B', 'OGFR', 'OSBPL2', 'PKN1', 'RAE1', 'RTF2', 'SENP2', 'SLC4A1AP', 'STAT1', 'TPD52L2', 'XPO5', 'YTHDF1', 'ZGPAT']

There are 3 shared correlations in the same negative direction
['GALNT10', 'PTPN3', 'TRIM2']

There are 0 shared correlations in the opposite direction
[]

There are 38 unique proteomic correlations not found in the transcriptome
There are 111 unique transcriptomic correlations not found in the proteome


### Unique Protein List for the Proteome and Transcriptome

In [1]:
unique_prot_htspt_wt = list()
for protein in proteomics_unique_tstat_2:
    unique_prot_htspt_wt.append(protein)

unique_prot_htspt_wt

NameError: name 'proteomics_unique_tstat_2' is not defined

In [25]:
unique_tran_htspt_wt = list()
for protein in transcriptomics_unique_tstat_2:
    unique_tran_htspt_wt.append(protein)

unique_tran_htspt_wt

['ABCF1',
 'ABCF3',
 'ABHD16B',
 'ABHD18',
 'ADRM1',
 'ALG2',
 'ANKHD1-EIF4EBP3',
 'ANKRD27',
 'ANXA2P1',
 'ARFGAP1',
 'ARFRP1',
 'ARSD',
 'ATP2C2',
 'BABAM1',
 'BAK1',
 'BBS4',
 'BYSL',
 'C9orf152',
 'CABLES2',
 'COLCA1',
 'CSTF1',
 'CYB5D2',
 'DCAF15',
 'DDX27',
 'DNAJC5',
 'DTX3L',
 'EPS8',
 'ERICH3',
 'FAAP24',
 'FNDC7',
 'FOXA2',
 'FSD1',
 'FTHL17',
 'FYTTD1',
 'GID8',
 'GMEB2',
 'GTPBP3',
 'IFIT1',
 'IL20RA',
 'KBTBD4',
 'KIAA1324',
 'KIAA2026',
 'KIF27',
 'KLHDC7A',
 'KPNA1',
 'LNP1',
 'LOC101927391',
 'LOC392232',
 'MANSC1',
 'MAP6D1',
 'MARVELD2',
 'MAST1',
 'MDM2',
 'MEIOB',
 'MGME1',
 'MIR17HG',
 'MRGBP',
 'MTG2',
 'NCBP2',
 'NGDN',
 'NOP56',
 'NPBWR2',
 'NPDC1',
 'OR5H2',
 'OR5H6',
 'PARL',
 'PDCD5',
 'PDRG1',
 'PEPD',
 'PFDN4',
 'PGR',
 'PGR-AS1',
 'PHKB',
 'PIK3R4',
 'PLA2G10',
 'PLPP2',
 'PRPF6',
 'PSMA7',
 'PSMD2',
 'RBM42',
 'RPS21',
 'RSAD2',
 'SBF2',
 'SECISBP2',
 'SENP5',
 'SGCB',
 'SMIM13',
 'SORBS2',
 'SPATA18',
 'SPDEF',
 'SS18L1',
 'SZRD1',
 'TBL1XR1',
 'TFG',
 

# TP53 Non-Hotspot Mutants Vs. Wildtype TP53

In [10]:
proteomics_nonhs_wt_tstat = {}
pcutoff = 0.05 / len(proteomics.columns)

for protein in proteomics.columns:
    if (protein != 'idx') and (protein != gene):
        p53_mutations_protein = CPTAC.compare_mutations(proteomics, protein, gene)
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        mutated = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == False]
        wt_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(mutated) < 2: continue
        ttest = scipy.stats.ttest_ind(mutated[protein], wt_tp53[protein])
        if ttest[1] <= pcutoff:
            proteomics_nonhs_wt_tstat[protein] = ttest[0]

In [16]:
transcriptomics_nonhs_wt_tstat = {}
pcutoff = 0.05 / len(transcriptomics.columns)

for protein in transcriptomics.columns:
    if (protein != 'idx') and (protein != gene):
        p53_mutations_protein = CPTAC.compare_mutations(transcriptomics, protein, gene)
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        mutated = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == False]
        wt_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(mutated) < 2: continue
        ttest = scipy.stats.ttest_ind(mutated[protein], wt_tp53[protein])
        if ttest[1] <= pcutoff:
            transcriptomics_nonhs_wt_tstat[protein] = ttest[0]

In [38]:
proteomics_shared_tstat_3 = {}
proteomics_unique_tstat_3 = {}

transcriptomics_shared_tstat_3 = {}
transcriptomics_unique_tstat_3 = {}

shared_identical_direction_pos_3 = list()
shared_identical_direction_neg_3 = list()
shared_opposite_direction_3 = list()


for protein in proteomics_nonhs_wt_tstat:
    if protein in transcriptomics_nonhs_wt_tstat:
        proteomics_shared_tstat_3[protein] = proteomics_nonhs_wt_tstat[protein]
        transcriptomics_shared_tstat_3[protein] = transcriptomics_nonhs_wt_tstat[protein]

for protein in proteomics_nonhs_wt_tstat:
    if protein not in transcriptomics_nonhs_wt_tstat:
        proteomics_unique_tstat_3[protein] = proteomics_nonhs_wt_tstat[protein]
        
for protein in transcriptomics_nonhs_wt_tstat:
    if protein not in proteomics_nonhs_wt_tstat:
        transcriptomics_unique_tstat_3[protein] = transcriptomics_nonhs_wt_tstat[protein]

for protein in proteomics_shared_tstat_3:
    if proteomics_shared_tstat_3[protein] > 0 and transcriptomics_shared_tstat_3[protein] > 0:
        shared_identical_direction_pos_3.append(protein)
    if proteomics_shared_tstat_3[protein] < 0 and transcriptomics_shared_tstat_3[protein] < 0:
        shared_identical_direction_neg_3.append(protein)
    if proteomics_shared_tstat_3[protein] < 0 and transcriptomics_shared_tstat_3[protein] > 0:
        shared_opposite_direction_3.append(protein)
    if proteomics_shared_tstat_3[protein] > 0 and transcriptomics_shared_tstat_3[protein] < 0:
        shared_opposite_direction_3.append(protein)
        
print('There are ' + str(len(shared_identical_direction_pos_3)) + ' shared correlations in the same positive direction')
print(shared_identical_direction_pos_3)

print('\n' + 'There are ' + str(len(shared_identical_direction_neg_3)) + ' shared correlations in the same negative direction')
print(shared_identical_direction_neg_3)

print('\n' + 'There are ' + str(len(shared_opposite_direction_3)) + ' shared correlations in the opposite direction')
print(shared_opposite_direction_3)

print('\n' + 'There are ' + str(len(proteomics_unique_tstat_3)) + ' unique proteomic correlations not found in the transcriptome')
print('There are ' + str(len(transcriptomics_unique_tstat_3)) + ' unique transcriptomic correlations not found in the proteome')        

There are 3 shared correlations in the same positive direction
['CDKAL1', 'LSG1', 'STAT2']

There are 3 shared correlations in the same negative direction
['KIF13B', 'SNX7', 'VPS13C']

There are 0 shared correlations in the opposite direction
[]

There are 18 unique proteomic correlations not found in the transcriptome
There are 83 unique transcriptomic correlations not found in the proteome


### Unique Protein List for the Proteome and Transcriptome

In [27]:
unique_prot_nonhs_wt = list()
for protein in proteomics_unique_tstat_3:
    unique_prot_nonhs_wt.append(protein)

unique_prot_nonhs_wt

['BIRC2',
 'COPS4',
 'FASTKD5',
 'FXR1',
 'GEMIN2',
 'GFM1',
 'GRB7',
 'GXYLT1',
 'HELZ2',
 'MTHFD2',
 'NGDN',
 'NOL10',
 'PRRC2A',
 'RBBP6',
 'SLC4A1AP',
 'TRMT6',
 'VAV2',
 'WDR75']

In [26]:
unique_tran_nonhs_wt = list()
for protein in transcriptomics_unique_tstat_3:
    unique_tran_nonhs_wt.append(protein)

unique_tran_nonhs_wt

['AADACL3',
 'ABCF1',
 'ABCF3',
 'ADAM18',
 'AEN',
 'ANKHD1-EIF4EBP3',
 'ANKLE1',
 'CCDC124',
 'CDK5RAP1',
 'CHD8',
 'CHRAC1',
 'CMTR1',
 'CSNK2A1',
 'CTCFL',
 'CYB5D2',
 'DERL1',
 'DHX16',
 'DIS3L',
 'DNAJB11',
 'DNMBP',
 'ECPAS',
 'ERI3',
 'FGF21',
 'FKBP8',
 'GGCX',
 'GID8',
 'GTF3C2',
 'HNF4G',
 'HPDL',
 'IDE',
 'IDH3B',
 'KLHDC3',
 'KPNA1',
 'LEMD2',
 'LINC01111',
 'LINC01980',
 'LINC01981',
 'LINC02548',
 'LOC100421746',
 'LOC101928978',
 'LOC399815',
 'MCM8',
 'MED20',
 'MIS18A',
 'MNX1',
 'MRGBP',
 'MRPL2',
 'MTBP',
 'MTG2',
 'NCBP2-AS2',
 'NLRP10',
 'NPBWR2',
 'NPDC1',
 'NSMCE2',
 'PARL',
 'PEPD',
 'POLDIP3',
 'PRDM9',
 'PRPF6',
 'RAE1',
 'RNF26',
 'RNF41',
 'RPP40',
 'RPRD1B',
 'RPS27L',
 'SENP2',
 'SENP5',
 'SHKBP1',
 'SIAE',
 'STX17',
 'SUMO3',
 'TAF11',
 'TBC1D22B',
 'TPD52L2',
 'TRIM13',
 'TRIM39',
 'TTC38',
 'WDR53',
 'ZFAT',
 'ZGPAT',
 'ZIC3',
 'ZNF679',
 'ZPBP2']